## Tools 

Allows models tpo connext with external functions

• Richer reponses by extending knowledge

• Ability to carry out actuions within the application

• Enhanced capabilities, like calculations

### Tool calling :

An LLM only generate Tokens! Our code calls the tool

### Common Use cases for tools

• Fetch data or add Knowledhge or context

• Take action, like booking a meeting

• Perform calculations or run code 

• Modify the UI

#### Two other ways to use tools that forms basis of Agentic AI:

1. A tool can be used to make another call to an LLM

2. A tool can be used to track a ToDo list and track progrss towards a goal

## Project:  Airline AL Assistant

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr                                                                                                                              

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [3]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be acacurate. If you don't know the answer, say so.
"""

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [16]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [17]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

### Building tool calling with SQLite Dtabase Integration

In [8]:
import sqlite3

In [49]:
price_DB = "prices.db"

with sqlite3.connect(price_DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

In [10]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(price_DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [ ]:
get_ticket_price("London")

In [23]:
def set_ticket_price(city, price):
    with sqlite3.connect(price_DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()

In [ ]:
get_ticket_price("Tokyo")

### Exercise : Create a way to add the ticket prices through chat, make booking and using efficient python libraries

Codes to add city and price details

In [18]:
set_price_function = {
    "name": "set_ticket_price",
    "description": "Add a city and the ticket price of that city",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "number",
                "description": "Ticket price of the destination_city",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": True
    }
}

In [54]:
tools.append({"type": "function", "function": set_price_function})

In [57]:
booking_function = {
    "name": "book_the_ticket",
    "description": "Book a ticket for a city based on user request",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "booking_date": {
                "type": "string",
                "description": "Date on which the user want to travel to destination_city",
            },
        },
        "required": ["destination_city", "booking_date"],
        "additionalProperties": True
    }
}

In [50]:
booking_DB = "booking.db"

with sqlite3.connect(booking_DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS booking (city TEXT, booking_date TEXT)')
    conn.commit()

In [53]:
def book_the_ticket(city, booking_date):
    try:
        with sqlite3.connect(booking_DB) as conn:
            cursor = conn.cursor()
            cursor.execute('INSERT INTO booking (city, booking_date) VALUES (?, ?)', (city.lower(), booking_date))
            conn.commit()
            return f"Ticket booked to {city} on {booking_date} Successfully"
    except Exception as e:
        return f"Booking failed: {str(e)}"

In [58]:
tools.append({"type": "function", "function": booking_function})

In [62]:
#Function to handle if one tool need to be accessed multiple times - optimized with dictionary lookup
def handle_tool_calls(message):
    def handle_get_price(args):
        city = args.get('destination_city')
        return get_ticket_price(city)

    def handle_set_price(args):
        city = args.get('destination_city')
        price = args.get('price')
        set_ticket_price(city, price)
        return "Details added successfully"

    def handle_book_ticket(args):
        city = args.get('destination_city')
        booking_date = args.get('booking_date')
        return book_the_ticket(city, booking_date)

    handlers = {
        "get_ticket_price": handle_get_price,
        "set_ticket_price": handle_set_price,
        "book_the_ticket": handle_book_ticket
    }

    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        content = handlers[tool_call.function.name](arguments)
        responses.append({
            "role": "tool",
            "content": content,
            "tool_call_id": tool_call.id
        })
    return responses

In [21]:
#If a tool need to be called mutiple times 
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_calls(message)
        messages.append(message)
        messages.extend(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(fn=chat).launch()